# **PRUEBAS 2D**

In [1]:
# IMPORT LIBRARIES

# Processing
import librosa
import librosa.display
import numpy as np
import random
from tqdm import tqdm
import cv2
from PIL import Image

# Visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import IPython.display as ipd

# Files
import os
import joblib
import pickle

# Machine Learning
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
import keras
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, AveragePooling1D
import tensorflow as tf

# ####### TEST ####### 
# Scipy
from scipy import signal
from scipy.io import wavfile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## LECTURA DE DATOS

In [3]:
AUDIO_DATA_PATH = 'data/'
GPATH = '/content/drive/My Drive/Master/Asignaturas/2 Cuatrimestre/Proyecto/Code/'
LPATH_DATA = 'SpeechEmotionRecognition/data/procesed/' 
LPATH_IMG = 'spectrograms/' 
SAMPLE_FILE = "03-01-01-01-01-01-01.wav"

# Maps
EMOTION_MAP = {0:'neutral', 1:'calm', 2:'happy', 3:'sad', 4:'angry', 5:'fear', 6:'disgust', 7:'surprise'}
INTENSITY_MAP = {1:'normal', 2:'strong'}

In [5]:
def load_spectograms_dataset(path, mfcc=True, mel=True, classes=EMOTION_MAP, verbose=True):
  '''
  Devuelve un dataframe con las imágenes generadas que se usarán tanto en el entrenamiento 
  como en test. Asume que en el path especificado existen esos archivos.
  
  Arguments:
  ----------
    path: str
      Directorio donde se encuentran las imagenes
    mfcc: boolean
      Especifica si se añaden los espectogramas MFCC al dataset
    mel: boolean
      Especifica si se añaden los espectogramas de MEL al dataset
    classes: dictionary
      Categorias a las que pertenecen las imagenes
    verbose: boolean
      Especifica si se activan los mensajes mientras se leen las imagenes

  Returns:
  ----------
  '''
  list_images = []
  labels = []
  if mfcc:
    mfcc_img, mfcc_lb = load_images(path + "mfcc/", EMOTION_MAP)
    list_images.extend(mfcc_img)
    labels.extend(mfcc_lb)

  if mel:
    mel_img, mel_lb = load_images(path + "mel/", EMOTION_MAP)
    list_images.extend(mel_img)
    labels.extend(mel_lb)

  return list_images, labels

def load_images(path, classes=EMOTION_MAP, verbose=True):

  '''
  Lee las imágenes que se cargarán desde un determinado directorio
  Arguments:
  ----------
    path: str
      Directorio donde se encuentran las imagenes
    classes: dictionary
      Categorias a las que pertenecen las imagenes
    verbose: boolean
      Especifica si se activan los mensajes mientras se leen las imagenes

  Returns:
  ----------
  '''
  list_images = []
  labels = []
  for index, emotion in classes.items():
    emodir = os.path.join(path, emotion)
    files = os.listdir(emodir)
    images = [file for file in files if file.endswith("jpg")]
    if verbose:
      print("Leídas {} espectogramas pertenecientes a {}".format(len(images), emotion))
    for image_name in images:
      image = cv2.imread(os.path.join(emodir, image_name))
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      img_arr = Image.fromarray(image, 'RGB')
      resized_img = img_arr.resize((227, 227)) # AlexNet format

      list_images.append(np.array(resized_img))
      labels.append(index)

  return list_images, labels




In [6]:
img_spectograms, labels = load_spectograms_dataset(GPATH + LPATH_IMG, True, False)
print("Se han leído {} imagenes en total".format(len(img_spectograms) ))

Leídas 96 espectogramas pertenecientes a neutral
Leídas 266 espectogramas pertenecientes a calm
Leídas 192 espectogramas pertenecientes a happy
Leídas 192 espectogramas pertenecientes a sad
Leídas 1920 espectogramas pertenecientes a angry
Leídas 192 espectogramas pertenecientes a fear
Leídas 192 espectogramas pertenecientes a disgust
Leídas 192 espectogramas pertenecientes a surprise
Se han leído 3242 imagenes en total


## PREPROCESADO DE LOS DATOS

In [7]:
images = np.array(img_spectograms)
labels = np.array(labels)

print("Dimensión de los datos: {}".format(images.shape))
print("Dimensión de las clases: {}".format(labels.shape))

Dimensión de los datos: (3242, 227, 227, 3)
Dimensión de las clases: (3242,)


In [8]:
# Barajamos de manera aleatoria tanto los datos como las clases
perm = np.random.permutation(len(images))
shuffle_img, shuffle_lb = images[perm], labels[perm]

# Normalización para facilitar la convergencia con CNN. Ajustar el recorrido de 0 a 1
data_images = shuffle_img.astype(np.float32)
labels = labels.astype(np.int32)
data_images = data_images/255.0

In [9]:
# División de los datos de entrenamiento y test
X, X_test, y, y_test = train_test_split(data_images, labels, test_size = 0.2)

# Comprobamos la dimensión de cada uno de los conjuntos
print("Dimension de datos de entrenamiento: {} y de datos de test: {}".format(X.shape, X_test.shape))
print("Dimension de las clases entrenamiento: {} y clases de test: {}".format(y.shape, y_test.shape))

Dimension de datos de entrenamiento: (2593, 227, 227, 3) y de datos de test: (649, 227, 227, 3)
Dimension de las clases entrenamiento: (2593,) y clases de test: (649,)


In [10]:
# One-hot encoding: convierto clases en un array de 0 y 1.

yn = to_categorical(y, len(EMOTION_MAP))
yn_test = to_categorical(y_test, len(EMOTION_MAP))

print("La clase {} categorizada: {}".format(EMOTION_MAP[2], yn[2]))

La clase happy categorizada: [0. 0. 0. 0. 1. 0. 0. 0.]


In [11]:
def alexNet_model(n_classes, input_shape):
  '''
  '''
  print("Modelo con entrada de datos con dimension: {} y {} categorias".format(input_shape, n_classes))
  model = Sequential()
  # 1 2D Conv
  model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding="valid",activation="relu",input_shape = input_shape ))
  model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
  model.add(BatchNormalization())
  # 2 2D Conv
  model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding="valid",activation="relu"))
  model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
  model.add(BatchNormalization())
  # 3 2D Conv
  model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))
  # 4 2D Conv
  model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))
  # 5 2D Conv
  model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))
  model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
  model.add(BatchNormalization())
  # Flatten
  model.add(Flatten())
  # 1 dense layer
  model.add(Dense(4096,input_shape = input_shape, activation="relu"))
  model.add(Dropout(0.4))
  model.add(BatchNormalization())

  #2 dense layer
  model.add(Dense(4096,activation="relu"))
  model.add(Dropout(0.4))
  model.add(BatchNormalization())

  #3 dense layer
  model.add(Dense(1000,activation="relu"))
  model.add(Dropout(0.4))
  model.add(BatchNormalization())

  # Salida softmax
  model.add(Dense(n_classes
                  ,activation="softmax"))

  return model

def CNN_multifeature():
  model = Sequential()
  model.add(Conv1D(256, 8, padding='same',input_shape=(66,1)))
  model.add(Activation('relu'))
  model.add(Conv1D(256, 8, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.6))
  model.add(Conv1D(128, 8, padding='same'))
  model.add(Activation('relu'))
  model.add(Conv1D(128, 8, padding='same'))
  model.add(Activation('relu'))
  model.add(Conv1D(128, 8, padding='same'))
  model.add(Activation('relu'))
  model.add(Conv1D(128, 8, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.6))
  model.add(Conv1D(64, 8, padding='same'))
  model.add(Activation('relu'))
  model.add(Conv1D(64, 8, padding='same'))
  model.add(Activation('relu'))
  model.add(Flatten())
  model.add(Dense(4096, activation='relu'))
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.6))
  model.add(Dense(16))
  model.add(Activation('softmax'))




In [ ]:
alexnet_model = alexNet_model(len(EMOTION_MAP), X.shape[1:])
# opt = keras.optimizers.SGD(lr = 0.001, momentum=0.9, decay=0.0)
opt = keras.optimizers.Adam(lr=0.0001)
# Compilamos el modelo
alexnet_model.compile(loss='categorical_crossentropy', 
              optimizer = opt, 
              metrics=['accuracy'])

# Entrenamos 
alexnet_model.fit(X, yn, batch_size=16, 
                        epochs=100)

In [ ]:
cnn_mult